In [1]:
import tensorflow as tf
import numpy as np
#import tf2jax

# Understanding DeepMind's DM21 model.

In [2]:
model = tf.saved_model.load('/Users/pablo.casares/Developer/q-data-dft/saved_model')
vars = model.variables
graph = model.graph
ops = graph.get_operations()

2022-11-15 12:05:39.248181: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


## The input features

The layer defining the inputs is the 27th operation in the graph.

In [3]:
print('len of ops', len(ops))
operation = ops[27]
print(operation.name)
print('Operation inputs:')
for i in operation.inputs:
    print(i)
print('Operation outputs:')
for o in operation.outputs:
    print(o)

# ops[:12]: module_fn___/default/Placeholder_i
# ops[12:26]: hub_wrapper/local_functional_v2/Const_i
# ops[26], ops[27]:
# hub_wrapper/local_functional_v2/SquashUnprocessedData/concat_grid_tensors/concat/axis
# hub_wrapper/local_functional_v2/SquashUnprocessedData/concat_grid_tensors/concat

len of ops 544
hub_wrapper/local_functional_v2/SquashUnprocessedData/concat_grid_tensors/concat
Operation inputs:
Tensor("module_fn___/default/Placeholder_7:0", shape=(None, 1), dtype=float32)
Tensor("module_fn___/default/Placeholder_8:0", shape=(None, 1), dtype=float32)
Tensor("module_fn___/default/Placeholder_4:0", shape=(None, 1), dtype=float32)
Tensor("module_fn___/default/Placeholder_5:0", shape=(None, 1), dtype=float32)
Tensor("module_fn___/default/Placeholder_6:0", shape=(None, 1), dtype=float32)
Tensor("module_fn___/default/Placeholder_9:0", shape=(None, 1), dtype=float32)
Tensor("module_fn___/default/Placeholder_10:0", shape=(None, 1), dtype=float32)
Tensor("module_fn___/default/Placeholder_2:0", shape=(None, 2), dtype=float32)
Tensor("module_fn___/default/Placeholder_3:0", shape=(None, 2), dtype=float32)
Tensor("hub_wrapper/local_functional_v2/SquashUnprocessedData/concat_grid_tensors/concat/axis:0", shape=(), dtype=int32)
Operation outputs:
Tensor("hub_wrapper/local_function

These Placeholder inputs are defined in the model signatures. 

In [4]:
model.signatures

_SignatureMap({'default': <ConcreteFunction pruned(tensor_dict$grid_coords, tensor_dict$grid_weights, tensor_dict$hfxa, tensor_dict$hfxb, tensor_dict$norm_grad_rho, tensor_dict$norm_grad_rho_a, tensor_dict$norm_grad_rho_b, tensor_dict$rho_a, tensor_dict$rho_b, tensor_dict$tau_a, tensor_dict$tau_b) at 0x1679FBF10>})

Warning: This order does not coincide to that indicated in the function [`_build_graph()`][def] from class `NeuralNumInt` in the `neural_numint.py` file from DeepMind repository.

[def]: https://github.com/deepmind/deepmind-research/blob/a318c69018c7064bd67f1d8cb84e54911d0d0d02/density_functional_approximation_dm21/density_functional_approximation_dm21/neural_numint.py#L252-L265

## The output features

This is a list of some of the most relevant output operations in the graph. We want to check how the output features are concatenated.

In [5]:
ops = graph.get_operations()
print('len of ops', len(ops))
for op in ops[450:470]:
    print(op.name)

len of ops 544
hub_wrapper/local_functional_v2/OutputLayer/linear/w
hub_wrapper/local_functional_v2/OutputLayer/linear/w/IsInitialized/VarIsInitializedOp
hub_wrapper/local_functional_v2/OutputLayer/linear/w/Assign
hub_wrapper/local_functional_v2/OutputLayer/linear/w/Read/ReadVariableOp
hub_wrapper/local_functional_v2/OutputLayer/linear/MatMul/ReadVariableOp
hub_wrapper/local_functional_v2/OutputLayer/linear/MatMul
hub_wrapper/local_functional_v2/OutputLayer/linear/b/Initializer/zeros
hub_wrapper/local_functional_v2/OutputLayer/linear/b
hub_wrapper/local_functional_v2/OutputLayer/linear/b/IsInitialized/VarIsInitializedOp
hub_wrapper/local_functional_v2/OutputLayer/linear/b/Assign
hub_wrapper/local_functional_v2/OutputLayer/linear/b/Read/ReadVariableOp
hub_wrapper/local_functional_v2/OutputLayer/linear/add/ReadVariableOp
hub_wrapper/local_functional_v2/OutputLayer/linear/add
hub_wrapper/local_functional_v2/OutputLayer/truediv/y
hub_wrapper/local_functional_v2/OutputLayer/truediv
hub_wrap

Operation 468 is specially relevant, as it multiplies the output of the last layer by the external features.

In [6]:
operation = ops[468]
print(operation.name)
print('Operation inputs:')
for i in operation.inputs:
    print(i)
print('Operation outputs:')
for o in operation.outputs:
    print(o)
print

hub_wrapper/local_functional_v2/OutputLayer/mul_3
Operation inputs:
Tensor("hub_wrapper/local_functional_v2/OutputLayer/mul_2:0", shape=(None, 3), dtype=float32)
Tensor("hub_wrapper/local_functional_v2/OutputLayer/concat:0", shape=(None, 3), dtype=float32)
Operation outputs:
Tensor("hub_wrapper/local_functional_v2/OutputLayer/mul_3:0", shape=(None, 3), dtype=float32)


<function print>

What is the input layer `Tensor("hub_wrapper/local_functional_v2/OutputLayer/concat:0", shape=(None, 3), dtype=float32)`?

In [7]:
concatops = []
for op in ops:
    if 'concat' in op.name: #hub_wrapper/local_functional_v2/OutputLayer/
        concatops.append(op)
op = concatops[3]
print(op.name)
print('Operation inputs:')
for i in op.inputs:
    print(i)
print('Operation outputs:')
for o in op.outputs:
    print(o)


hub_wrapper/local_functional_v2/OutputLayer/concat
Operation inputs:
Tensor("hub_wrapper/local_functional_v2/OutputLayer/add:0", shape=(None, 1), dtype=float32)
Tensor("hub_wrapper/local_functional_v2/OutputLayer/add_1:0", shape=(None, 2), dtype=float32)
Tensor("hub_wrapper/local_functional_v2/OutputLayer/concat/axis:0", shape=(), dtype=int32)
Operation outputs:
Tensor("hub_wrapper/local_functional_v2/OutputLayer/concat:0", shape=(None, 3), dtype=float32)


We want to find out whether `Tensor("hub_wrapper/local_functional_v2/OutputLayer/add_1:0", shape=(None, 2), dtype=float32)` makes reference to the Hartree Fock energy

In [8]:
concatops = []
for op in ops:
    if 'add_1' in op.name: #hub_wrapper/local_functional_v2/OutputLayer/
        concatops.append(op)
op = concatops[8]
print(op.name)
print('Operation inputs:')
for i in op.inputs:
    print(i)
print('Operation outputs:')
for o in op.outputs:
    print(o)

hub_wrapper/local_functional_v2/OutputLayer/add_1
Operation inputs:
Tensor("module_fn___/default/Placeholder_2:0", shape=(None, 2), dtype=float32)
Tensor("module_fn___/default/Placeholder_3:0", shape=(None, 2), dtype=float32)
Operation outputs:
Tensor("hub_wrapper/local_functional_v2/OutputLayer/add_1:0", shape=(None, 2), dtype=float32)


Presumably this means that the output of the model is really the Hartree Fock energy, summed for each spin. The 2 features are the two values of $\omega$ considered.

# Other operations

In [9]:
def find_op(ops, name):
    l = []
    for op in ops:
        if name in op.name:
            l.append(op)
    return l

In [10]:
adds = find_op(ops, 'add')
print(len(adds))
for add in adds[-1:]:
    print(add.name)
    print('Operation inputs:')
    for i in add.inputs:
        print(i)
    print('Operation outputs:')
    for o in add.outputs:
        print(o)

46
hub_wrapper/local_functional_v2/OutputLayer/linear/add
Operation inputs:
Tensor("hub_wrapper/local_functional_v2/OutputLayer/linear/MatMul:0", shape=(None, 3), dtype=float32)
Tensor("hub_wrapper/local_functional_v2/OutputLayer/linear/add/ReadVariableOp:0", shape=(3,), dtype=float32)
Operation outputs:
Tensor("hub_wrapper/local_functional_v2/OutputLayer/linear/add:0", shape=(None, 3), dtype=float32)


## Inspecting the model

In [11]:
i = 433
print('Name', ops[i].name)
print('Inputs', ops[i].inputs)
print('Outputs',ops[i].outputs)

Name hub_wrapper/local_functional_v2/OutputLayer/Pow
Inputs (<tf.Tensor 'module_fn___/default/Placeholder_7:0' shape=(None, 1) dtype=float32>, <tf.Tensor 'hub_wrapper/local_functional_v2/OutputLayer/Pow/y:0' shape=() dtype=float32>)
Outputs [<tf.Tensor 'hub_wrapper/local_functional_v2/OutputLayer/Pow:0' shape=(None, 1) dtype=float32>]


In [12]:
ops = graph.get_operations()
ops

[<tf.Operation 'module_fn___/default/Placeholder' type=Placeholder>,
 <tf.Operation 'module_fn___/default/Placeholder_1' type=Placeholder>,
 <tf.Operation 'module_fn___/default/Placeholder_2' type=Placeholder>,
 <tf.Operation 'module_fn___/default/Placeholder_3' type=Placeholder>,
 <tf.Operation 'module_fn___/default/Placeholder_4' type=Placeholder>,
 <tf.Operation 'module_fn___/default/Placeholder_5' type=Placeholder>,
 <tf.Operation 'module_fn___/default/Placeholder_6' type=Placeholder>,
 <tf.Operation 'module_fn___/default/Placeholder_7' type=Placeholder>,
 <tf.Operation 'module_fn___/default/Placeholder_8' type=Placeholder>,
 <tf.Operation 'module_fn___/default/Placeholder_9' type=Placeholder>,
 <tf.Operation 'module_fn___/default/Placeholder_10' type=Placeholder>,
 <tf.Operation 'hub_wrapper/local_functional_v2/Const' type=Const>,
 <tf.Operation 'hub_wrapper/local_functional_v2/Const_1' type=Const>,
 <tf.Operation 'hub_wrapper/local_functional_v2/Const_2' type=Const>,
 <tf.Operati

In [13]:
x = np.random.normal(size=(1, 11)).astype(np.float32)
# jax_func, jax_params = tf2jax.convert(model, np.zeros_like(x))

In [14]:
j = 470
for op in ops[470:]:
    if op.inputs != () or op.outputs != ():
        print(j,op.name)
        print('Operation inputs:')
        for i in op.inputs:
            print(i)
        print('Operation outputs:')
        for o in op.outputs:
            print(o)
        print('-----------------------------------------\n')
    j += 1

470 hub_wrapper/local_functional_v2/OutputLayer/Sum
Operation inputs:
Tensor("hub_wrapper/local_functional_v2/OutputLayer/mul_3:0", shape=(None, 3), dtype=float32)
Tensor("hub_wrapper/local_functional_v2/OutputLayer/Sum/reduction_indices:0", shape=(), dtype=int32)
Operation outputs:
Tensor("hub_wrapper/local_functional_v2/OutputLayer/Sum:0", shape=(None,), dtype=float32)
-----------------------------------------

471 hub_wrapper/local_functional_v2/Const_15
Operation inputs:
Operation outputs:
Tensor("hub_wrapper/local_functional_v2/Const_15:0", shape=(1,), dtype=float32)
-----------------------------------------

472 save/filename/input
Operation inputs:
Operation outputs:
Tensor("save/filename/input:0", shape=(), dtype=string)
-----------------------------------------

473 save/filename
Operation inputs:
Tensor("save/filename/input:0", shape=(), dtype=string)
Operation outputs:
Tensor("save/filename:0", shape=(), dtype=string)
-----------------------------------------

474 save/Const

In [15]:
i = 88
print('Name', ops[i].name)
print('Inputs', ops[i].inputs)
print('Outputs',ops[i].outputs)

Name hub_wrapper/local_functional_v2/MLP/ResidualBlock/linear/add
Inputs (<tf.Tensor 'hub_wrapper/local_functional_v2/MLP/ResidualBlock/linear/MatMul:0' shape=(None, 256) dtype=float32>, <tf.Tensor 'hub_wrapper/local_functional_v2/MLP/ResidualBlock/linear/add/ReadVariableOp:0' shape=(256,) dtype=float32>)
Outputs [<tf.Tensor 'hub_wrapper/local_functional_v2/MLP/ResidualBlock/linear/add:0' shape=(None, 256) dtype=float32>]


In [16]:
op = ops[11]
op.__dict__

{'_original_op': None,
 '_graph': <tensorflow.python.framework.func_graph.FuncGraph at 0x167510520>,
 '_c_op': <tensorflow.python.client._pywrap_tf_session.TF_Operation at 0x167555ef0>,
 '_inputs_val': (),
 '_device_code_locations': [],
 '_colocation_code_locations': {},
 '_control_flow_context': None,
 '_gradient_function': None,
 '_is_stateful': False,
 '_outputs': [<tf.Tensor 'hub_wrapper/local_functional_v2/Const:0' shape=() dtype=float32>],
 '_id_value': 12}

In [17]:
model.__dict__

{'restore': <function tensorflow.python.saved_model.load_v1_in_v2._EagerSavedModelLoader.load.<locals>.<lambda>(path)>,
 '_self_setattr_tracking': True,
 '_self_unconditional_checkpoint_dependencies': [TrackableReference(name=initializer, ref=<tensorflow.python.saved_model.load_v1_in_v2._Initializer object at 0x1679df910>),
  TrackableReference(name=asset_paths, ref=ListWrapper([])),
  TrackableReference(name=signatures, ref=_SignatureMap({'default': <ConcreteFunction pruned(tensor_dict$grid_coords, tensor_dict$grid_weights, tensor_dict$hfxa, tensor_dict$hfxb, tensor_dict$norm_grad_rho, tensor_dict$norm_grad_rho_a, tensor_dict$norm_grad_rho_b, tensor_dict$rho_a, tensor_dict$rho_b, tensor_dict$tau_a, tensor_dict$tau_b) at 0x1679FBF10>})),
  TrackableReference(name=variables, ref=ListWrapper([<tf.Variable 'hub_wrapper/local_functional_v2/SquashUnprocessedData/linear/w:0' shape=(11, 256) dtype=float32, numpy=
  array([[ 0.4960697 , -0.02498221,  0.34590778, ..., -0.5886391 ,
           0.

In [18]:
graph.__dict__.keys()

dict_keys(['_lock', '_group_lock', '_nodes_by_id', '_next_id_counter', '_nodes_by_name', '_version', '_names_in_use', '_stack_state_is_thread_local', '_thread_local', '_graph_device_function_stack', '_default_original_op', '_control_flow_context', '_graph_control_dependencies_stack', '_collections', '_seed', '_attr_scope_map', '_op_to_kernel_label_map', '_gradient_override_map', '_gradient_function_map', '_finalized', '_functions', '_graph_def_versions', '_building_function', '_graph_colocation_stack', '_unfeedable_tensors', '_unfetchable_ops', '_handle_feeders', '_handle_readers', '_handle_movers', '_handle_deleters', '_graph_key', '_last_loss_reduction', '_is_loss_scaled_by_optimizer', '_container', 'experimental_acd_manager', '_add_control_dependencies', '_op_def_cache', '_bcast_grad_args_cache', '_reduced_shape_cache', '_c_graph', 'name', 'inputs', 'outputs', 'control_outputs', 'control_captures', 'structured_input_signature', 'structured_outputs', '_weak_variables', '_watched_vari

In [19]:
ops = graph.get_operations()

In [20]:
#model.prune(feeds=[ops[0].name], fetches=[ops[1].name])

In [21]:
ops[0].__dict__.keys()


dict_keys(['_original_op', '_graph', '_c_op', '_inputs_val', '_device_code_locations', '_colocation_code_locations', '_control_flow_context', '_gradient_function', '_is_stateful', '_outputs', '_id_value'])

In [22]:
ops[27]._inputs_val #= [tf.constant(np.random.normal(size=(1, 11)).astype(np.float32))]

(<tf.Tensor 'module_fn___/default/Placeholder_7:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'module_fn___/default/Placeholder_8:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'module_fn___/default/Placeholder_4:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'module_fn___/default/Placeholder_5:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'module_fn___/default/Placeholder_6:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'module_fn___/default/Placeholder_9:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'module_fn___/default/Placeholder_10:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'module_fn___/default/Placeholder_2:0' shape=(None, 2) dtype=float32>,
 <tf.Tensor 'module_fn___/default/Placeholder_3:0' shape=(None, 2) dtype=float32>,
 <tf.Tensor 'hub_wrapper/local_functional_v2/SquashUnprocessedData/concat_grid_tensors/concat/axis:0' shape=() dtype=int32>)

In [23]:
'''import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()'''
tf.compat.v1.disable_eager_execution()

sess = tf.compat.v1.Session(graph=graph)

t2 = graph.get_tensor_by_name('module_fn___/default/Placeholder_2:0')
t3 = graph.get_tensor_by_name('module_fn___/default/Placeholder_3:0')
t4 = graph.get_tensor_by_name('module_fn___/default/Placeholder_4:0')
t5 = graph.get_tensor_by_name('module_fn___/default/Placeholder_5:0')
t6 = graph.get_tensor_by_name('module_fn___/default/Placeholder_6:0')
t7 = graph.get_tensor_by_name('module_fn___/default/Placeholder_7:0')
t8 = graph.get_tensor_by_name('module_fn___/default/Placeholder_8:0')
t9 = graph.get_tensor_by_name('module_fn___/default/Placeholder_9:0')
t10 = graph.get_tensor_by_name('module_fn___/default/Placeholder_10:0')

np2 = np.random.normal(size=(1, 2)).astype(np.float32)
np3 = np.random.normal(size=(1, 2)).astype(np.float32)
np4 = np.random.normal(size=(1, 1)).astype(np.float32)
np5 = np.random.normal(size=(1, 1)).astype(np.float32)
np6 = np.random.normal(size=(1, 1)).astype(np.float32)
np7 = np.random.normal(size=(1, 1)).astype(np.float32)
np8 = np.random.normal(size=(1, 1)).astype(np.float32)
np9 = np.random.normal(size=(1, 1)).astype(np.float32)
np10 = np.random.normal(size=(1, 1)).astype(np.float32)

feed_dict = {t2: np2, t3: np3, t4: np4, t5: np5, t6: np6, t7: np7, t8: np8, t9: np9, t10: np10}

with sess.as_default():

    ops[27].run(session=sess, feed_dict = feed_dict)
    a = ops[27].outputs[0].eval(session=sess, feed_dict = feed_dict)
    print(a)

[[ 1.6284426   0.09642576  1.2672021   0.214064   -0.65343285 -0.3835043
   0.9824206   0.11205453 -0.56288195  0.03396611  0.8546302 ]]


In [24]:
ops[27].outputs

[<tf.Tensor 'hub_wrapper/local_functional_v2/SquashUnprocessedData/concat_grid_tensors/concat:0' shape=(None, 11) dtype=float32>]

In [25]:
ops[27].outputs

[<tf.Tensor 'hub_wrapper/local_functional_v2/SquashUnprocessedData/concat_grid_tensors/concat:0' shape=(None, 11) dtype=float32>]